In [53]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import joblib
import warnings
warnings.filterwarnings("ignore")

In [54]:
SEED = 2017
np.random.seed(SEED)

data = pd.read_csv('../Data/ratings_xs.csv')

In [55]:
data.head()

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


In [56]:
#idx = np.random.permutation(150)
X = data[['movie_id', 'user_id']]
y = data.rating

#### Building an ensemble
Instantiating a fully specified ensemble is straightforward and requires three steps: first create the instance, second add the intermediate layers, and finally the meta estimator.

In [57]:
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC

In [58]:
def build_ensemble(incl_meta, propagate_features=None):
    if propagate_features:
        n = len(propagate_features)
        propagate_features_1 = propagate_features
        propagate_features_2 = [i for i in range(n)]
    else:
        propagate_features_1 = propagate_features_2 = None
        
    estimators = [RandomForestClassifier(random_state=seed), SVC()]
    
    ensemble = SuperLearner()
    ensemble.add(estimators, propagate_features=propagate_features_1)
    ensemble.add(estimators, propagate_features=propagate_features_2)

    if incl_meta:
        ensemble.add_meta(LogisticRegression())
    return ensemble

In [60]:
base = build_ensemble(False)
base.fit(X, y)
pred = base.predict(X)[:2]
print("Input to meta learner :\n %r" % pred)

KeyboardInterrupt: 

In [45]:
print("Prediction score: %.3f" % accuracy_score(preds, y[75:]))


Prediction score: 0.338


#### Multi-layer ensembles
With each call to the `add` method, another layer is added to the ensemble. Note that all ensembles are sequential in the order layers are added. For instance, in the above example, we could add a second layer as follows.

In [46]:
ensemble = SuperLearner(scorer=accuracy_score, random_state=seed)

# Build the first layer
ensemble.add([RandomForestClassifier(random_state=seed), LogisticRegression()])

# Build the second layer
ensemble.add([LogisticRegression(), SVC()])

# Attach the final meta estimator
ensemble.add_meta(SVC())

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=9787, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...d8f620>)],
   n_jobs=-1, name='group-11', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=2017, sample_size=20,
       scorer=<function accuracy_score at 0x1a1dd8f620>, shuffle=False,
       verbose=False)

In [47]:
ensemble.fit(X[:75], y[:75])
preds = ensemble.predict(X[75:])
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                   score-m  score-s  ft-m  ft-s  pt-m  pt-s
layer-1  logisticregression           0.41     0.06  0.01  0.01  0.01  0.00
layer-1  randomforestclassifier       0.37     0.05  0.03  0.01  0.00  0.00
layer-2  logisticregression           0.45     0.02  0.00  0.00  0.00  0.00
layer-2  svc                          0.45     0.02  0.00  0.00  0.00  0.00



#### The scoring function

In [48]:
from mlens.metrics import make_scorer

In [49]:
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)

#### A simple evaluation
Before throwing preprocessing into the mix, let’s see how to evaluate a set of estimator. First, we need a list of estimator and a dictionary of parameter distributions that maps to each estimator. The estimators should be put in a list, either as is or as a named tuple ((name, est)). If you don’t name the estimator, the Evaluator will automatically name the model as the class name in lower case. This name must be the key in the parameter dictionary.

In [50]:
from mlens.model_selection import Evaluator 
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint

In [51]:
# Name the estimators
ests = [('gnb', GaussianNB()), ('knn', KNeighborsClassifier())]

# Mapping parameters to these / Since gnb doesn't have any parameters we can just skip it 
pars = {'n_neighbors': randint(2, 20)}
params = {'knn': pars}

We can now run an evaluation over these estimators and parameter distributions by calling the fit method.



In [52]:
evaluator = Evaluator(accuracy_scorer, cv=10, random_state=seed, verbose=1)
evaluator.fit(X, y, ests, params, n_iter=10)

Launching job


KeyboardInterrupt: 

The full history of the evaluation can be found in `cv_results`. To compare models with their best parameters, we can pass the `results` attribute to a pandas.DataFrame or print it as a table. We use `m` to denote mean values and `s` to denote standard deviation across folds for brevity. Note that the timed prediction is for the training set, for comparability with training time.

In [ ]:
print("Score comparison with best params founds:\n\n%r" % evaluator.results)


#### Preprocessing
Next, suppose we want to compare the models across a set of preprocessing pipelines. To do this, we first need to specify a dictionary of preprocessing pipelines to run through. Each entry in the dictionary should be a list of transformers to apply sequentially.

In [ ]:
from mlens.preprocessing import Subset
from sklearn.preprocessing import StandardScaler

In [ ]:
# Map preprocessing cases through a dictionary
preprocess_cases = {'none': [],
                    'sc': [StandardScaler()],
                    'sub': [Subset([0, 1])]
                    }

The fit methods determines automatically whether there is any preprocessing or any estimator jobs to run, so all we need to do is specify the arguments we want to be processed. If a previous preprocessing job was fitted, those pipelines are stored and will be used for subsequent estimator fits.

This can be helpful if the preprocessing is time-consuming, for instance if the preprocessing pipeline is an ensemble itself. All ensembles implement a `transform` method that, in contrast to the predict method, regenerates the predictions made during the fit``call. More precisely, the ``transform method uses the estimators fitted with cross-validation to construct predictions, whereas the `predict` method uses the final estimators fitted on all data. This allows us use ensembles as preprocessing steps that mimicks how that ensemble would produce predictions for a subsequent meta learner or layer. Since fitting large ensembles is highly time-consuming, fixing the lower layers as preprocessing input is highly valuable for tuning the higher layers and / or the final meta learner. 

In [ ]:
evaluator.fit(X, y, preprocessing=preprocess_cases)


#### Model Selection across preprocessing pipelines
To evaluate the same set of estimators across all pipelines with the same parameter distributions, there is no need to take any heed of the preprocessing pipeline, just carry on as in the simple case:

In [ ]:
evaluator.fit(X, y, ests, params, n_iter=10)
print("\nComparison across preprocessing pipelines:\n\n%r" % evaluator.results)
